In [ ]:
import os
import pandas as pd
import numpy as np
from IPython.display import display
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans
import logging
import math
import numpy as np
import torch
from torch.nn import ModuleList
from functools import partial
from typing import List, Tuple, Dict, Optional, Any
from torchvision.transforms import Compose, RandomApply
from torchvision.transforms import functional as F
from torchvision.transforms.transforms import _setup_angle, _check_sequence_input
from torch import Tensor
from collections import defaultdict, deque
from pathlib import Path
from torch import nn
from PIL import ImageFilter, ImageOps, Image, ImageDraw

import os
# 设定实验结果的目录 (请修改为你的实际路径)
result_dir = "/home/yuhaowang/project/FMBC/finetune/outputs/BRACS_Fine"

# 需要展示的评估指标
evaluation_metrics = ['val_bacc', 'val_weighted_f1', 'val_macro_auroc']

# 你希望的模型顺序（从上到下）
desired_order = [
    "UNI", "CONCH", "Virchow","Gigapath_Tile",'Gigapath',"CHIEF_Tile","TITAN","FMBC"  # 请修改为你的模型名称
]

all_results = []

# 遍历目录中的所有模型文件夹
for model_name in os.listdir(result_dir):
    model_path = os.path.join(result_dir, model_name, "summary.csv")
    
    # 检查是否存在 summary.csv 文件
    if os.path.isfile(model_path):
        df = pd.read_csv(model_path)

        # 计算均值和标准差
        summary_stats = {"Model": model_name}
        for metric in evaluation_metrics:
            if metric in df.columns:
                mean_val = np.mean(df[metric])
                std_val = np.std(df[metric], ddof=1)  # 样本标准差
                summary_stats[metric] = f"{mean_val:.3f}±{std_val:.4f}"

        # 添加到列表
        all_results.append(summary_stats)

# 转换为 DataFrame
final_result_df = pd.DataFrame(all_results)

# 按照提供的模型顺序排序
final_result_df['sort_order'] = final_result_df['Model'].apply(lambda x: desired_order.index(x) if x in desired_order else len(desired_order))
#delete the model not in desired_order
final_result_df = final_result_df[final_result_df['sort_order']!=len(desired_order)]
final_result_df = final_result_df.sort_values(by='sort_order').drop(columns=['sort_order'])
final_result_df.style.hide(axis="index")
# 在 Jupyter Notebook 中美观显示
display(final_result_df)




In [ ]:
import os
data_path = '/home/yuhaowang/data/processed_data'
embedding_path ='/data1/embedding'
for dataset in os.listdir(data_path):
    print(dataset,'has',len(os.listdir(os.path.join(data_path, dataset, 'output'))),'slides')
    dataset_embedding_path = os.path.join(embedding_path, dataset)
    if not os.path.exists(dataset_embedding_path):
        print(dataset, 'has no embedding')
        print('---------------------------')

        continue
    for model in os.listdir(dataset_embedding_path):
        print(model,'has',len(os.listdir(os.path.join(dataset_embedding_path, model))),'slides processed')
    print('---------------------------')
    


In [ ]:
import  h5py
def read_assets_from_h5( h5_path: str) -> tuple:
    '''Read the assets from the h5 file'''
    assets = {}
    attrs = {}
    with h5py.File(h5_path, 'r') as f:
        for key in f.keys():
            assets[key] = f[key][:]
            if f[key].attrs is not None:
                attrs[key] = dict(f[key].attrs)
    return assets, attrs

data_dir = '/data1/embedding/TCGA-BRCA'
for model in os.listdir(data_dir):
    embedding_path=os.path.join(data_dir,model)
    test_case = os.listdir(embedding_path)[0]
    test_data,_ = read_assets_from_h5(os.path.join(embedding_path,test_case))
    print(test_data.keys())
    print(model)
    print(test_data['features'].shape)
    